In [1]:
# from vision import Vision
from globalPlanning import GlobalPlanning
from localPlanning import LocalPlanning
from thymioControl import ThymioControl
from kalman import Kalman
from plotter import Plotter

import numpy as np
import math
import time

In [2]:
!pip3 install tdmclient --upgrade

In [3]:
import tdmclient.notebook
await tdmclient.notebook.start()

NodeLockError: Node lock error (current status: busy)

In [ ]:
import time

@tdmclient.notebook.sync_var
def motor_go(left,right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def motor_stop():
    global motor_left_target,motor_right_target
    motor_left_target = 0
    motor_right_target = 0
    
@tdmclient.notebook.sync_var
def sensor_data():
    global prox_horizontal
    return prox_horizontal.copy()

@tdmclient.notebook.sync_var
def leds_off():
    global leds_top, leds_bottom_left, leds_bottom_right, leds_circle
    leds_top = [0, 0, 0]
    leds_bottom_left = [0, 0, 0]
    leds_bottom_right = [0, 0, 0]
    leds_circle = [0, 0, 0, 0, 0, 0, 0, 0]

def leds_go_crazy():
    for i in range(8):
        leds_1()
        time.sleep(0.1)
        leds_2()
        time.sleep(0.1)
        leds_3()
        time.sleep(0.1)

@tdmclient.notebook.sync_var
def leds_1():
    global leds_top, leds_bottom_left, leds_bottom_right, leds_circle
    leds_top = [32, 0, 0]
    leds_bottom_left = [0, 32, 0]
    leds_bottom_right = [0, 0, 32]
    leds_circle = [32, 32, 32, 32, 0, 0, 0, 0]

@tdmclient.notebook.sync_var
def leds_2():
    global leds_top, leds_bottom_left, leds_bottom_right, leds_circle
    leds_top = [0, 32, 0]
    leds_bottom_left = [0, 0, 32]
    leds_bottom_right = [32, 0, 0]
    leds_circle = [0, 0, 0, 0, 32, 32, 32, 32]

@tdmclient.notebook.sync_var
def leds_3():
    global leds_top, leds_bottom_left, leds_bottom_right, leds_circle
    leds_top = [0, 0, 32]
    leds_bottom_left = [32, 0, 0]
    leds_bottom_right = [0, 32, 0]
    leds_circle = [32, 0, 32, 0, 32, 0, 32, 0]

In [ ]:
# vision = Vision()
globalPlanning = GlobalPlanning()
localPlanning = LocalPlanning()
thymio = ThymioControl()
plotter = Plotter()
filter = Kalman()

In [ ]:
matrix = np.zeros((20, 30))
matrix[1:6, 9:11] = -1; matrix[11, 1] = -1; matrix[10:17, 17:26] = -1
start = (1, 1)
goal = (19, 28)
plotter.set_map(matrix, start, goal)

plotter.plot_map()

NameError: name 'np' is not defined

In [ ]:
motor_go(100, 100)

In [ ]:
motor_stop()

In [ ]:
x_trajectory = []
y_trajectory = []
kalman_positions = []

dt = 0.15

iter = 1

goal = False

map = matrix

# initial position and angle
position = np.array(start)
goal_pos = goal
angle = 0

thymio.set_timestep(dt)

# leds_off()

while not goal:
    print("Position: ", position)
    # print("position 0: ", position[0])
    # print("position 1: ", position[1])
    print("Angle: ", angle)

    if (iter == 1):
        print("First iteration")

        thymio.set_pose(position, angle)

        print("Computing path")
        # print("Map shape: ", map.shape)
        path = globalPlanning.dijkstra(map, tuple(position), goal_pos)
        print("Path: ", path)
        thymio.set_path(path)
        plotter.set_map(map, position, goal_pos)
        plotter.plot_path(thymio.get_path_cells())
        print("Path: ", thymio.get_path_cells())
        filter.initialize_position(thymio.get_position()[0], thymio.get_position()[1], angle)
        filter.set_lastKalman_time()

    # check if camera is obstructed
    if position is not None:
        thymio.update_pose(position, angle)
        x_trajectory.append(float(position[0]))
        y_trajectory.append(float(position[1]))
        # camera measuerement that will then be used for the Kalman filter
        measurement = np.array([thymio.get_position()[0], thymio.get_position()[1], angle])
        # filter.kalman_update(measurement)
        # print("Kalman update: ", filter.get_state())
    
    # get the state from the Kalman filter
    # the result will depend on the mode of the filter
    x, y, angle = filter.get_state()
    kalman_position = [x, y]
    kalman_positions.append(position)
    print("Kalman position used: ", kalman_position)
    print("Kalman angle used: ", angle)

    # check if the robot is detecting an obstacle
    # tmclient function to get the proximity sensors
    prox = sensor_data()
    if (localPlanning.is_obstacle_avoidance(prox)):
        # move with local planning until the robot is not back on the path
        wl, wr = localPlanning.obstacle_avoidance(prox)
        v, w = thymio.inverseDifferentialDrive(wl, wr)
    else:
        # move with global planning
        v, w, wl, wr, goal = thymio.move(kalman_position, angle)

    print("Speed: ", thymio.convert_speed_cells(v), w)
    print("Motor commands: ", wl, wr)

    # update the Kalman filter
    filter.kalman_prediction(wl, wr)
    print("Kalman prediction: ", filter.get_state())

    # tmdclient function to move the motors
    wl = int(wl)
    wr = int(wr)

    if abs(w) < 1e-6:
        # treat as a straight line
        position[0] = kalman_position[0] + v * dt * math.cos(angle)
        position[1] = kalman_position[1] + v * dt * math.sin(angle)
    else:
        # use the circular motion equations
        position[0] = kalman_position[0] + (v / w) * (math.sin(angle + w * dt) - math.sin(angle))
        position[1] = kalman_position[1] - (v / w) * (math.cos(angle + w * dt) - math.cos(angle))


    position[0] = thymio.mm_to_cells(position[0])
    position[1] = thymio.mm_to_cells(position[1])

    print("calculated position: ", position)

    c_angle = (angle + w*dt) % (2 * math.pi)
    print("calculated angle: ", c_angle)
    
    print("Trajectory: ", x_trajectory, y_trajectory)
    plotter.plot_trajectory(x_trajectory, y_trajectory)

    print("wl, wr: ", wl, wr)
    motor_go(wl, wr)
    
    
    # sleep for a while
    iter += 1
    time.sleep(dt)

motor_stop()
leds_go_crazy()
leds_off()

NameError: name 'matrix' is not defined